### Test how many images in ImageNette are segmented with more than 100 segments

In [1]:
# prepare images
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import random
random.seed(42)
import os, json


# test_imgs_idx = random.sample(range(0, len(ImgeNet)), test_imgs_num)
imgExtension = ["png", "jpeg", "JPEG", "jpg"] #Image Extensions to be chosen from
allImages = list()
def chooseRandomImage(directory="/media/hongbo/Experiment/Hongbo/ImageNet/imagenette-320px/val"):
    for img in os.listdir(directory): #Lists all files
        ext = img.split(".")[-1]
        if (ext in imgExtension):
            allImages.append(img)
    # choice = random.randint(0, len(allImages) - 1)
    # chosenImage = random.choices(allImages, k = test_imgs_num)
    random.shuffle(allImages)
    chosenImage = list(set(allImages))
    # print('Number of available images:', len(chosenImage))
    # chosenImage = allImages[choice] #Do Whatever you want with the image file
    return [os.path.join(directory, img) for img in chosenImage]

randomImage = chooseRandomImage()
print('Number of available images:', len(randomImage))

/home/hongbo/anaconda3/envs/hongbo_lipex/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of available images: 3925


In [2]:
# prepare SegmentAnything
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "/mnt/b432dc15-0b9a-4f76-9076-5bf99fe91d74/Hongbo/segment-anything/checkpoint/sam_vit_h_4b8939.pth"
model_type = "vit_h"
device = "cuda:0"
print(torch.cuda.get_device_name(device=device))
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
mask_generator = SamAutomaticMaskGenerator(sam)

NVIDIA GeForce RTX 3090


In [3]:
# apply SegmentAnything on images with more than 100 segments are found
from tqdm import tqdm
valid_images, images_segments_num, segmented_images = [], [], []
for image_path in tqdm(randomImage):
    image = cv2.imread(image_path)
    image_name = image_path.split('/')[-1]
    image = cv2.resize(image, (256, 256))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # print("Image shape:", image.shape)
    masks = mask_generator.generate(image)
    masks = sorted(masks, key=(lambda x: x['area']))

    LIPEx_segments = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
    for idx in range(len(masks))[::-1]:
        cur_mask = masks[idx]['segmentation']
        LIPEx_segments[cur_mask] = idx
    # print(LIPEx_segments)
    # print(len(np.unique(LIPEx_segments)), np.unique(LIPEx_segments))
    # np.save(os.path.join('ImageNette_Segments', image_name.replace('.JPEG', 'txt')), LIPEx_segments, fmt='%d')
    np.save(os.path.join('ImageNette_Segments', image_name.replace('.JPEG', 'npy')), LIPEx_segments)
    num_seg = len(np.unique(LIPEx_segments))
    segmented_images.append(image_path)
    with open('ImageNette/ImageNette_Segmented_files.txt', 'a') as sf:
         sf.write('{},{}\n'.format(image_name, num_seg))
    images_segments_num.append(num_seg)
    if num_seg > 99:
           valid_images.append(image_path)
           with open('ImageNette/valid_images.txt', 'a') as f:
               f.write(image_name+'\n')
    # if len(valid_images) == 200:
    #     break


100%|██████████| 3925/3925 [1:52:24<00:00,  1.72s/it]


In [6]:
# find unsegmented images
with open('ImageNette/ImageNette_Segmented_files.txt', 'r') as f:
    segmented_files = f.readlines()
    processed_images = [os.path.join('/media/hongbo/Experiment/Hongbo/ImageNet/imagenette-320px/val', line.split(',')[0]) for line in segmented_files]
    print('Number of processed images:', len(processed_images))

unprocessed_images = list(set(randomImage) - set(processed_images))
print('Number of unprocessed images:', len(unprocessed_images))

Number of processed images: 2251
Number of unprocessed images: 1674


In [8]:
# apply SegmentAnything on images: continue segmenting the rest of images
from tqdm import tqdm
valid_images, images_segments_num, segmented_images = [], [], []
for image_path in tqdm(unprocessed_images):
    image = cv2.imread(image_path)
    image_name = image_path.split('/')[-1]
    image = cv2.resize(image, (224, 224))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    # print("Image shape:", image.shape)
    masks = mask_generator.generate(image)
    masks = sorted(masks, key=(lambda x: x['area']))

    LIPEx_segments = np.zeros((image.shape[0], image.shape[1]), dtype=np.uint8)
    for idx in range(len(masks))[::-1]:
        cur_mask = masks[idx]['segmentation']
        LIPEx_segments[cur_mask] = idx
    # print(LIPEx_segments)
    # print(len(np.unique(LIPEx_segments)), np.unique(LIPEx_segments))
    # np.save(os.path.join('ImageNette_Segments', image_name.replace('.JPEG', 'txt')), LIPEx_segments, fmt='%d')
    np.save(os.path.join('ImageNette_Segments', image_name.replace('.JPEG', 'npy')), LIPEx_segments)
    num_seg = len(np.unique(LIPEx_segments))
    segmented_images.append(image_path)
    with open('ImageNette/ImageNette_Segmented_files.txt', 'a') as sf:
         sf.write('{},{}\n'.format(image_name, num_seg))
    images_segments_num.append(num_seg)
    if num_seg > 99:
           valid_images.append(image_path)
           with open('ImageNette/valid_images.txt', 'a') as f:
               f.write(image_name+'\n')


100%|██████████| 1674/1674 [47:27<00:00,  1.70s/it]


In [2]:
# prepare images
import numpy as np
import matplotlib.pyplot as plt
import random
random.seed(42)
import os, json


# test_imgs_idx = random.sample(range(0, len(ImgeNet)), test_imgs_num)
imgExtension = ["png", "jpeg", "JPEG", "jpg"] #Image Extensions to be chosen from
allImages = list()
def chooseRandomImage(directory="/media/hongbo/Experiment/Hongbo/ImageNet/imagenette-320px/val"):
    for img in os.listdir(directory): #Lists all files
        ext = img.split(".")[-1]
        if (ext in imgExtension):
            allImages.append(img)
    chosenImage = list(set(allImages))
    for img in chosenImage:
        with open('ImageNette/images.txt', 'a') as f:
            f.write(img+'\n')
    # return [os.path.join(directory, img) for img in chosenImage]
randomImage = chooseRandomImage()
# print('Number of available images:', len(randomImage))
# print(randomImage[:3])

Number of available images: 3925
['/media/hongbo/Experiment/Hongbo/ImageNet/imagenette-320px/val/n03425413_14672.JPEG', '/media/hongbo/Experiment/Hongbo/ImageNet/imagenette-320px/val/n03394916_46941.JPEG', '/media/hongbo/Experiment/Hongbo/ImageNet/imagenette-320px/val/n03000684_1692.JPEG']


In [4]:
# prepare images
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2
import random
random.seed(42)
import os, json

# test_imgs_idx = random.sample(range(0, len(ImgeNet)), test_imgs_num)
imgExtension = ["png", "jpeg", "JPEG", "jpg"] #Image Extensions to be chosen from
allImages = list()
test_imgs_num = 200
def chooseRandomImage(directory="/media/hongbo/Experiment/Hongbo/ImageNet/imagenette-320px/val"):
    for img in os.listdir(directory): #Lists all files
        ext = img.split(".")[-1]
        if (ext in imgExtension):
            allImages.append(img)
    choice = random.randint(0, len(allImages) - 1)
    chosenImage = random.choices(allImages, k = test_imgs_num)
    return chosenImage
    # chosenImage = allImages[choice] #Do Whatever you want with the image file
    # return [os.path.join(directory, img) for img in chosenImage]

randomImage = chooseRandomImage()
print(randomImage[:3])
# print('Number of chosen images:', len(randomImage))

['n01440764_2921.JPEG', 'n03445777_832.JPEG', 'n01440764_1951.JPEG']


In [16]:
import random
random.seed(42)

img_directory="/media/hongbo/Experiment/Hongbo/ImageNet/imagenette-320px/val"
segs_path = '/mnt/b432dc15-0b9a-4f76-9076-5bf99fe91d74/Hongbo/LIPEx/segment-anything/notebooks/ImageNette_Segments'
def chooseRandomImage(
        valid_files="/mnt/b432dc15-0b9a-4f76-9076-5bf99fe91d74/Hongbo/LIPEx/segment-anything/notebooks/ImageNette/valid_images.txt"
        ):
    with open(valid_files, 'r') as f:
        chosenImage = f.readlines()
    chosenImage = [img.strip() for img in chosenImage]
    print('Number of available images:', len(chosenImage))
    return chosenImage

randomImage_path = chooseRandomImage()
random.shuffle(randomImage_path)
print(randomImage_path[:10])

Number of available images: 378
['n03394916_29940.JPEG', 'n03394916_47211.JPEG', 'n03417042_1770.JPEG', 'n03000684_13270.JPEG', 'n03028079_12732.JPEG', 'n03394916_26391.JPEG', 'n03417042_25212.JPEG', 'n03000684_16151.JPEG', 'n03425413_13831.JPEG', 'n03000684_2472.JPEG']
